In [25]:
import pandas as pd

df = pd.read_csv("results/results_ok_1.csv")
df.drop(['GldStd', 'PPIN', 'Refs'], axis=1).groupby(by=['Method']).mean()

df[df.PPIN == 'BIM'].drop(['GldStd', 'PPIN', 'Refs'], axis=1).groupby(by=['Method']).mean()



,Predicts,Precision,Recall,F1-score,F2-score,AUC-PR,MMR,Sensitivity,PPP,Accuracy,F-Match,Separation
Method,,,,,,,,,,,,
CUBCO+,1679.55,0.383829,0.889706,0.534834,0.484942,0.440587,0.800672,0.862978,0.608445,0.724157,0.939788,0.312506
ClusterOne,190.50,0.932472,0.603350,0.732444,0.427324,0.483381,0.462726,0.826355,0.668715,0.743347,0.689542,0.646027
P5COMP,20956.55,0.834392,0.902369,0.866633,0.593353,0.637345,0.856026,0.926421,0.599025,0.744105,0.948366,0.085500
PC2P,261.50,0.934392,0.798162,0.860808,0.545324,0.637047,0.640324,0.893311,0.710163,0.796441,0.881904,0.791374
